In [17]:
import matplotlib.pyplot as plt
import numpyro
from numpyro import handlers
import numpyro.distributions as dist
import numpyro_stein.stein.kernels as kernels
from numpyro.infer.util import Predictive
from numpyro_stein.stein.autoguides import AutoDelta
from numpyro_stein.util import init_with_noise
from numpyro_stein.stein.stein import SVGD
from numpyro_stein.distributions.normal_mixture_distribution import NormalMixture
from numpyro_stein.stein.kernels import SteinKernel
from numpyro.infer import ELBO
from numpyro.infer.util import init_to_value
from tqdm import tqdm
import seaborn as sns
import os
import jax
import jax.numpy as np
from random import randint
import scipy.io

In [18]:
# From http://theoval.cmp.uea.ac.uk/matlab/default.html
data = scipy.io.loadmat('data/benchmarks.mat')

In [19]:
datasets = {k: {'train': {'input': v['x'][0, 0][v['train'][0, 0][0, :] - 1], 'class': (v['t'][0, 0][v['train'][0, 0][0, :] - 1] == 1).astype('float')[:, 0]}, 'test': {'input': v['x'][0, 0][v['test'][0, 0][0, :] - 1], 'class': (v['t'][0, 0][v['test'][0, 0][0, :] - 1] == 1).astype('float')[:, 0]}} for k, v in data.items() if not str.startswith(k, "__") and not k == 'benchmarks' and v['x'][0, 0].shape[0] > 500}

In [38]:
rng_key = jax.random.PRNGKey(randint(0, int(1e6)))
num_iterations = 3000
num_particles = 100

In [52]:
def model(data, classes=None):
    alpha = numpyro.sample('alpha', dist.InverseGamma(concentration=1.0, rate=0.01))
    w = numpyro.sample('w', dist.Normal(loc=np.zeros(data.shape[1]), scale=alpha))
    with numpyro.plate('data', data.shape[0]):
        with handlers.scale(scale_factor=1/data.shape[0]):
            return numpyro.sample('x', dist.Bernoulli(logits=data @ w), obs=classes)

In [53]:
guide = AutoDelta(model, init_strategy=init_with_noise(init_to_value({'x': -10.}), noise_scale=1.0))

In [54]:
def test_accuracy(rng_key, testset, params):
    def single_test_accuracy(rng_key, testset, params):
        guide_trace = handlers.trace(handlers.substitute(guide, params)).get_trace(testset['input'])
        model_trace = handlers.trace(handlers.replay(handlers.seed(model, rng_key), guide_trace)).get_trace(testset['input'])
        accuracy = np.count_nonzero(model_trace['x']['value'] == testset['class']) / testset['input'].shape[0] * 100
        return accuracy
    accs = []
    for i in range(num_particles):
        ps = {k: param[i] for k, param in params.items()}
        accs.append(single_test_accuracy(rng_key, testset, ps))
    return np.mean(np.stack(accs))

In [55]:
for name, dataset in datasets.items():
    print(name)
    svgd = SVGD(model, guide, numpyro.optim.Adagrad(step_size=0.1), ELBO(),
                kernels.RBFKernel(), num_stein_particles=num_particles, num_loss_particles=1)
    svgd_state = svgd.init(rng_key, dataset['train']['input'], dataset['train']['class'])
    pbar = tqdm(range(num_iterations))
    for i in pbar:
        svgd_state, loss = svgd.update(svgd_state, dataset['train']['input'], dataset['train']['class'])
        pbar.set_description("SVGD: {}".format(loss))
    print(test_accuracy(svgd_state.rng_key, dataset['test'], svgd.get_params(svgd_state)))

banana
SVGD: -7.60223388671875: 100%|██████████| 3000/3000 [05:44<00:00,  8.71it/s]
50.33959
diabetis
SVGD: 25.506385803222656:  12%|█▏        | 354/3000 [00:41<05:13,  8.45it/s]


KeyboardInterrupt: 